<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/475_MO_agent_execution_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Why This Design Builds Executive Trust

From a leadership perspective, this execution model is reassuring because:

* Agent actions are permissioned
* Results follow known templates
* Failures are explicit
* Behavior is consistent run-to-run

There is no “AI magic” here — only controlled execution.

That makes the system:

* Easier to test
* Easier to audit
* Easier to scale
* Easier to explain

---

# Agent Execution Utilities — Architectural Explanation

## What This Component Does

This module defines the **execution bridge** between the Mission Orchestrator’s plan and the agents that carry out individual tasks.

In the MVP, execution is intentionally **rule-based and simulated**. Instead of calling real agents or APIs, the system:

* Selects an appropriate agent based on declared capabilities
* Simulates task execution using predefined result templates
* Produces a standardized execution record for downstream tracking, governance, and reporting

This allows you to validate **orchestration logic, control flow, and accountability** without being distracted by implementation details.

---

## Why This Matters Architecturally

Most agent systems conflate:

* Planning
* Agent selection
* Execution
* Results

This module keeps those concerns cleanly separated.

By simulating execution while preserving real execution semantics, you gain:

* Deterministic behavior
* Repeatable testing
* Clear audit trails
* A smooth path to production later

In other words, you’re testing the *system*, not the tools.

---

## Core Responsibilities of This Module

### 1. Controlled Agent Selection

Agent selection is **explicit, explainable, and capability-driven**.

Rather than relying on model inference or dynamic reasoning, the system:

* Looks up which agents are allowed to perform a task
* Selects from those agents based on availability
* Fails cleanly if no valid option exists

This ensures:

* Task routing decisions can be defended
* Agent permissions are enforced
* Capability gaps are visible immediately

From a governance standpoint, this answers:

> “Why did this agent perform this task?”

---

### 2. Execution Without Hidden Intelligence

The execution function deliberately avoids:

* LLM calls
* Heuristics
* Implicit reasoning

Instead, it:

* Loads a predefined result template
* Returns a structured, predictable output
* Simulates success or failure deterministically

This is a **design strength**, not a limitation.

It allows you to validate:

* State transitions
* KPI updates
* Human-in-the-loop logic
* Reporting accuracy

…without introducing randomness or cost.

---

### 3. Standardized Execution Records

Every task execution produces a **fully structured execution record**.

Each record captures:

* Task identity
* Agent identity
* Execution timing
* Status and result
* Approval requirements
* Errors when applicable

This turns execution into **operational data**, not ephemeral output.

That data is what powers:

* Progress tracking
* KPI evaluation
* Audit trails
* Post-mission reporting

---

### 4. Graceful Failure Handling

If no agent is available for a task, the system:

* Does not crash
* Does not guess
* Does not improvise

It returns a clear failure record explaining *why* execution could not proceed.

This makes operational gaps explicit and actionable:

* Missing agents
* Incorrect capability mappings
* Resource constraints

Failures become insights, not surprises.

---

### 5. Clear Separation Between “What” and “How”

This module does not decide:

* What the mission is
* What the plan should be
* Whether a task is allowed to proceed

It simply:

* Executes the next approved step
* Records the outcome faithfully

That separation ensures:

* Planning logic remains centralized
* Execution logic remains reusable
* Governance logic remains enforceable

---

## How This Fits Into the Mission Orchestrator

In the larger system, this module is called only when:

* A task is ready
* Dependencies are satisfied
* Governance rules allow execution

It acts as a **pure execution layer**, feeding results back into the orchestrator state, where:

* Progress is updated
* KPIs are recalculated
* Human approvals may be triggered
* Reports are eventually generated

This keeps the orchestration loop clean and predictable.

---


## MVP Discipline (and Why It’s the Right Choice)

By simulating execution now, you’ve avoided a common trap:

> Building brittle integrations before validating orchestration value.

This approach lets you:

* Prove the architecture
* Validate KPIs and reporting
* Stress-test governance logic
* Demonstrate business value early

When you later swap in real agents or APIs, the **contract stays the same** — only the implementation changes.

That’s exactly how enterprise systems should evolve.

---

## Bottom Line

This utilities module is the **execution backbone** of your MVP.

It transforms plans into observable action while preserving:

* Determinism
* Transparency
* Accountability
* Upgrade flexibility

You’re not simulating “AI behavior” — you’re simulating **operational execution**.

That distinction is what makes this orchestrator credible.



In [ ]:
"""Agent Execution Utilities

Simulate agent execution for MVP. Rule-based, no actual agent calls.
Uses result templates from agent_execution_results.json.
"""

import json
from pathlib import Path
from typing import Dict, Any, Optional
from datetime import datetime
from agents.mission_orchestrator.utilities.data_loading import load_agent_result_template


def select_agent_for_task(
    task_id: str,
    capabilities_lookup: Dict[str, list],
    agents_lookup: Dict[str, Dict[str, Any]]
) -> Optional[Dict[str, Any]]:
    """
    Select an agent for a task based on capabilities.

    MVP: Simple selection - pick first available agent capable of the task.
    """
    capable_agent_ids = capabilities_lookup.get(task_id, [])

    if not capable_agent_ids:
        return None

    # MVP: Select first available agent
    for agent_id in capable_agent_ids:
        agent = agents_lookup.get(agent_id)
        if agent and agent.get("status") == "available":
            return agent

    return None


def execute_agent_task(
    task: Dict[str, Any],
    agent: Dict[str, Any],
    execution_context: Optional[Dict[str, Any]] = None,
    data_dir: str = "agents/data"
) -> Dict[str, Any]:
    """
    Execute a task using an agent (MVP: rule-based simulation).

    For MVP, this simulates agent execution by:
    1. Loading result template from agent_execution_results.json
    2. Returning the template result (simulating successful execution)

    In production, this would make actual agent API calls.
    """
    task_id = task.get("task_id")
    agent_id = agent.get("agent_id")
    agent_name = agent.get("name")

    # Load result template
    result_template = load_agent_result_template(task_id, data_dir)

    # MVP: Simulate execution by using template
    # In production, this would be an actual agent call
    if result_template:
        result = result_template.copy()
    else:
        # Fallback: Create basic result structure
        result = {
            "status": "completed",
            "message": f"Task {task_id} executed by {agent_name}",
            "timestamp": datetime.now().isoformat()
        }

    # Create execution record
    execution_record = {
        "task_id": task_id,
        "task": task.get("task"),
        "agent_id": agent_id,
        "agent_name": agent_name,
        "status": result.get("status", "completed"),
        "start_time": datetime.now().isoformat(),
        "end_time": datetime.now().isoformat(),
        "duration_minutes": task.get("estimated_duration_minutes", 0),
        "result": result,
        "requires_human_approval": task.get("requires_human_approval", False),
        "error": result.get("error") if result.get("status") == "failed" else None
    }

    return execution_record


def execute_next_task(
    task_queue: list,
    executed_tasks: list,
    capabilities_lookup: Dict[str, list],
    agents_lookup: Dict[str, Dict[str, Any]],
    execution_context: Optional[Dict[str, Any]] = None,
    data_dir: str = "agents/data"
) -> Optional[Dict[str, Any]]:
    """
    Execute the next ready task from the queue.

    Returns execution record if task was executed, None if no tasks ready.
    """
    if not task_queue:
        return None

    # Get first task from queue (already ordered by dependencies)
    task = task_queue[0]
    task_id = task.get("task_id")

    # Select agent for task
    agent = select_agent_for_task(task_id, capabilities_lookup, agents_lookup)

    if not agent:
        # No agent available for this task
        return {
            "task_id": task_id,
            "task": task.get("task"),
            "status": "failed",
            "error": f"No available agent found for task {task_id}",
            "start_time": datetime.now().isoformat(),
            "end_time": datetime.now().isoformat()
        }

    # Execute task
    execution_record = execute_agent_task(
        task,
        agent,
        execution_context,
        data_dir
    )

    return execution_record
